# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
cities_weather_df = pd.read_csv('../WeatherPy/Output Data/City_Data.csv')
del cities_weather_df['Unnamed: 0']
cities_weather_df.head(3)

,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,rikitea,-23.12,-134.97,67.24,59,39,14.07
1,sangar,63.92,127.47,74.88,49,93,9.89
2,makokou,0.57,12.86,67.12,94,66,2.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Store latitude and longitude in coordinates
coordinates = cities_weather_df[['Latitude', 'Longitude']]
map_weight = cities_weather_df['Humidity (%)']

In [16]:
# Plot Heatmap
fig = gmaps.figure()

In [17]:
# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=map_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# My kind of weather Condition: 
## 77 =< Maximum temperature >= 68.
## Cloudiness < 30%
## Humidity < 70%

ideal_weather_df = cities_weather_df.loc[('Cloudiness (%)' < 30) & ('Humidity (%)' < 70),('Max Temperature (F)' <= 77) & ('Max Temperature (F)' >= 68), :]
ideal_weather_df
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number like 10.
## WHAT DOES THIS MEAN? ^





TypeError: '<' not supported between instances of 'str' and 'int'

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# store what variable into hotel_df?

hotel_df

# Use the Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
target_type = "hotel"

radius = 5000

# rewrite params dictionary --- wait waitt
params = {
    "location": coordinates,
    "types": target_type,
    "radius": radius,
    "key": gkey
}


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
